In [1]:
# -*- coding: utf-8 -*-
train_data_txt = "../training/msr_training.txt"
init_train_data = []
with open(train_data_txt,encoding='gbk') as trt:
    init_train_data = trt.readlines()
all_data = "".join(init_train_data)
train_data_split = [line.strip().split() for line in init_train_data]

print(train_data_split[0:5])
print(len(train_data_split))

[['“', '人们', '常', '说', '生活', '是', '一', '部', '教科书', '，', '而', '血', '与', '火', '的', '战争', '更', '是', '不可多得', '的', '教科书', '，', '她', '确实', '是', '名副其实', '的', '‘', '我', '的', '大学', '’', '。'], ['“', '心', '静', '渐', '知', '春', '似', '海', '，', '花', '深', '每', '觉', '影', '生', '香', '。'], ['“', '吃', '屎', '的', '东西', '，', '连', '一', '捆', '麦', '也', '铡', '不', '动', '呀', '？'], ['他', '“', '严格要求', '自己', '，', '从', '一个', '科举', '出身', '的', '进士', '成为', '一个', '伟大', '的', '民主主义', '者', '，', '进而', '成为', '一', '位', '杰出', '的', '党外', '共产主义', '战士', '，', '献身', '于', '崇高', '的', '共产主义', '事业', '。'], ['“', '征', '而', '未', '用', '的', '耕地', '和', '有', '收益', '的', '土地', '，', '不准', '荒芜', '。']]
86924


In [2]:
# 读取字符词典文件
char_vocab_path = "../training/char_vocabs.txt"
special_words = ['<PAD>', '<UNK>'] # 特殊词表示
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs
print(char_vocabs[0:10])

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

['<PAD>', '<UNK>', '!', '#', '%', '&', "'", '(', ')', '*']


In [3]:
# "BMES"标记的标签
label2idx = {"S": 0,
             "B": 1, 
             "M": 2,
             "E": 3
             }
# 索引和BMES标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

In [4]:
# 读取data
def data_list(path, vocab2idx, label2idx):
    with open(train_data_txt,encoding='gbk') as trt:
        lines = trt.readlines()
    datas, labels= [], []
    for line in lines: # line是一个字符串
        mline = line.strip().split() #mline是个词列表
        sent, tag = [],[]
        for i,word in enumerate(mline):
            if len(word)==1:
                sent.append(word)
                tag.append("S")
            else:
                s_word = ["M" for i in word]
                s_word[0] = "B"
                s_word[-1] = "E"
                sent+=list(word)
                tag+=s_word

        sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent]
        tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag]
        datas.append(sent_ids)
        labels.append(tag_ids)

    return datas, labels

In [5]:
datas, labels = data_list(train_data_txt, vocab2idx, label2idx)
print(len(datas),len(labels))
print(datas[5])
print([idx2vocab[idx] for idx in datas[5]])
print(labels[5])
print([idx2label[idx] for idx in labels[5]])

86924 86924
[122, 5800, 6506, 563, 2572, 240, 3007, 2530, 6209, 6440, 6802, 3007, 2530, 3903, 2102, 2065, 6209, 6440, 182]
['“', '这', '首', '先', '是', '个', '民', '族', '问', '题', '，', '民', '族', '的', '感', '情', '问', '题', '。']
[0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0]
['S', 'S', 'B', 'E', 'S', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'S']


In [55]:
snum = 70000
train_datas = datas[0:snum]
train_labels = labels[0:snum]
test_datas = datas[snum:]
test_labels = labels[snum:]
print(train_labels[0:10])

[[0, 1, 3, 0, 0, 1, 3, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 2, 2, 3, 0, 1, 2, 3, 0, 0, 1, 3, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 2, 2, 3, 1, 3, 0, 0, 1, 3, 1, 3, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 2, 2, 3, 0, 0, 1, 3, 1, 3, 0, 0, 1, 3, 0, 1, 3, 1, 2, 2, 3, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 2, 2, 3, 1, 3, 0], [0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 0, 0, 1, 3, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 3, 0], [0, 1, 3, 1, 3, 1, 3, 0, 1, 2, 3, 1, 3, 0, 1, 2, 3, 1, 2, 2, 3, 0, 1, 3, 0, 1, 3, 0], [0, 1, 3, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0], [0, 0, 1, 3, 0, 1, 3, 1, 3, 0, 1, 3, 0, 0, 0, 0]]


In [56]:
from keras.utils import to_categorical

batch_size = 128

import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()
from crf_keras import CRF
from keras.layers import Dense, Embedding, Conv1D, Input
from keras.models import Model # 这里我们学习使用Model型的模型
import keras.backend as K # 引入Keras后端来自定义loss，注意Keras模型内的一切运算
                          # 必须要通过Keras后端完成，比如取对数要用K.log不能用np.log

embedding_size = 128
EPOCHS = 2
MAX_LEN=100
CLASS_NUMS=5
print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
test_datas = sequence.pad_sequences(test_datas, maxlen=MAX_LEN)
test_labels = sequence.pad_sequences(test_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', test_datas.shape)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)#.reshape((70000,100, 5))
test_labels = keras.utils.to_categorical(test_labels, CLASS_NUMS)#.reshape((16924,100, 5))
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)



sequence = Input(shape=(None,), dtype='int32') # 建立输入层，输入长度设为None
embedding = Embedding(len(char_vocabs)+1,
                      embedding_size,
                     )(sequence) # 去掉了mask_zero=True
cnn = Conv1D(128, 3, activation='relu', padding='same')(embedding)
cnn = Conv1D(128, 3, activation='relu', padding='same')(cnn)
cnn = Conv1D(128, 3, activation='relu', padding='same')(cnn) # 层叠了3层CNN

crf = CRF(True) # 定义crf层，参数为True，自动mask掉最后一个标签
tag_score = Dense(5)(cnn) # 变成了5分类，第五个标签用来mask掉
tag_score = crf(tag_score) # 包装一下原来的tag_score

model = Model(inputs=sequence, outputs=tag_score)
model.summary()

model.compile(loss=crf.loss, # 用crf自带的loss
              optimizer='adam',
              metrics=[crf.accuracy] # 用crf自带的accuracy
             )
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)


model.save_weights("../model/CNN_seg_weight.h5")
model.save("../model/CNN_seg.h5")

padding sequences
x_train shape: (70000, 100)
x_test shape: (16924, 100)
trainlabels shape: (70000, 100)
testlabels shape: (16924, 100)
trainlabels shape: (70000, 100, 5)
testlabels shape: (16924, 100, 5)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         880000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         49280     
____________________________________________________

In [57]:
BATCH_SIZE = 64
score = model.evaluate(test_datas, test_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

16924/16924 [==============================] - 8s 481us/step
['loss', 'accuracy']
[3.797654495710347, 0.9708165916616656]


In [58]:
def max_in_dict(d): # 定义一个求字典中最大值的函数
    key,value = list(d.items())[0]
    for i,j in list(d.items())[1:]:
        if j > value:
            key,value = i,j
    return key,value

def viterbi(nodes, trans): # viterbi算法，跟前面的HMM一致
    paths = nodes[0] # 初始化起始路径
    for l in range(1, len(nodes)): # 遍历后面的节点
        paths_old,paths = paths,{}
        for n,ns in nodes[l].items(): # 当前时刻的所有节点
            max_path,max_score = '',-1e10
            for p,ps in paths_old.items(): # 截止至前一时刻的最优路径集合
                score = ns + ps + trans[p[-1]+n] # 计算新分数
                if score > max_score: # 如果新分数大于已有的最大分
                    max_path,max_score = p+n, score # 更新路径
            paths[max_path] = max_score # 储存到当前时刻所有节点的最优路径
#     print(paths)
    return max_in_dict(paths)


def cut(s, trans): # 分词函数，也跟前面的HMM基本一致
    if not s: # 空字符直接返回
        return []
    # 字序列转化为id序列。注意，经过我们前面对语料的预处理，字符集是没有空格的，
    # 所以这里简单将空格的id跟句号的id等同起来
    sent_ids = np.array([[vocab2idx.get(c, 0) if c != ' ' else vocab2idx[u'。']
                          for c in s]])
    probas = model.predict(sent_ids)[0] # 模型预测
    nodes = [dict(zip('SBME', i)) for i in probas[:, :4]] # 只取前4个
    nodes[0] = {i:j for i,j in nodes[0].items() if i in 'BS'} # 首字标签只能是b或s
    nodes[-1] = {i:j for i,j in nodes[-1].items() if i in 'ES'} # 末字标签只能是e或s
    tags = viterbi(nodes, trans)[0]
    result = [s[0]]
    for i,j in zip(s[1:], tags[1:]):
        if j in 'BS': # 词的开始
            result.append(i)
        else: # 接着原来的词
            result[-1] += i
    return result

In [59]:
t_ = model.get_weights()[-1][:4,:4] # 从训练模型中取出最新得到的转移矩阵
# t_ = np.array([[ 0.2551752,   1.3712068,  -1.969512,   -0.94762987],
#       [-1.3502188,  -0.11775374,  0.6775522,   2.1938992 ],
#       [-2.6458318,  -0.72990716,  1.816428,    0.97933584],
#       [ 0.73843807,  1.0811784,  -1.9546679,  -0.7160042 ]])

trans = {}
for i in 'SBME':
    for j in 'SBME':
        trans[i+j] = t_[label2idx[i], label2idx[j]]
result = cut("储存到当前时刻所有节点的最优路径.", trans)
print(result)

['储存', '到', '当前', '时刻', '所有', '节点', '的', '最', '优路', '径', '.']


In [60]:
t_ = model.get_weights()[-1][:4,:4] 
print(t_)

[[ 0.54433304  0.31405345 -1.4422159  -1.2881045 ]
 [-1.971681   -2.0412054   0.89083886  1.6226453 ]
 [-0.87709403 -1.8920363   1.4513828   0.46186703]
 [ 1.2752982   0.657804   -1.4817071  -1.950337  ]]


[[7.3534518e-01 1.4596057e-01 7.9223752e-02 3.8083300e-02]
 [6.3772547e-01 1.5958913e-01 3.3011988e-02 1.6931488e-01]
 [4.3838363e-02 8.7163794e-01 6.9017991e-02 1.5349535e-02]
 [3.9679520e-02 3.2943442e-02 4.7815818e-02 8.7947863e-01]
 [7.2943561e-02 8.5752940e-01 1.4630562e-02 5.4879654e-02]
 [2.9200837e-01 9.3740760e-04 1.6958232e-03 7.0535827e-01]
 [7.3800108e-04 9.9867404e-01 1.0722456e-04 4.8074606e-04]
 [1.7609337e-04 9.1550837e-04 3.6390350e-04 9.9854457e-01]
 [9.9484384e-01 1.5055639e-04 2.2830433e-04 4.7772792e-03]
 [1.5624326e-03 9.9740249e-01 1.0211968e-03 1.3845533e-05]
 [1.1583652e-03 9.8706312e-07 7.0626709e-05 9.9876994e-01]
 [9.9908912e-01 8.8811549e-04 1.9136700e-05 3.5502653e-06]
 [9.7760838e-01 1.9476980e-02 2.5318420e-04 2.6613877e-03]
 [9.2452709e-03 9.6629715e-01 2.2554133e-02 1.9029514e-03]
 [1.1260571e-02 1.0427126e-05 1.0238566e-04 9.8862666e-01]
 [9.9973387e-01 2.2136088e-04 3.9861356e-05 5.0519270e-06]
 [5.7004488e-01 4.2899254e-01 1.2667493e-04 8.3547673e-0

[[ 1.609243   -0.60307544 -0.5321975  -1.350059  ]
 [ 2.1912084  -0.50220716 -1.9891578  -1.5365913 ]
 [-1.2706817   0.44745424  0.21643111 -0.7846817 ]
 [-0.79834855 -2.2858722  -0.7888813   2.3905082 ]
 [-0.4829742   1.3791542  -0.72924024 -1.4593649 ]
 [-0.21598284 -4.392511   -1.1265367   3.3144002 ]
 [-2.0698733   2.1803555  -0.8679491  -0.8824459 ]
 [-1.6855692  -1.5468001  -1.2390535   2.8636303 ]
 [ 3.8489604  -4.0107965  -0.8362013  -1.6430526 ]
 [ 0.26872087  1.424837   -0.645475   -3.6561563 ]
 [ 0.7524922  -3.4886906  -1.6522187   2.5781956 ]
 [ 4.5253954  -1.041991    0.11846626 -5.8860846 ]
 [ 4.114442   -5.469349   -1.738103   -0.07103527]
 [-1.6244731   2.3750694  -0.2490682  -3.1855345 ]
 [ 1.146612   -5.647822   -2.6322265   4.5583663 ]
 [ 5.8469195  -1.4046049   0.5016388  -8.909219  ]
 [ 2.5169191   1.220266   -2.5986495  -2.5697556 ]
 [ 0.16200602 -1.077709   -0.27034223  0.06126173]]
最优状态路径为: [1. 3. 1. 3. 4. 2. 1. 4. 2. 4. 2. 4. 2. 4. 2. 4. 1. 3.]


[[ 0.2551752  -1.3502188  -2.6458318   0.73843807]
 [ 1.3712068  -0.11775374 -0.72990716  1.0811784 ]
 [-1.969512    0.6775522   1.816428   -1.9546679 ]
 [-0.94762987  2.1938992   0.97933584 -0.7160042 ]]
